# 0.1 imports

In [39]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt

from statsmodels.tsa.api import SimpleExpSmoothing

# 0.2 helpers functions

# 0.3 load data

In [40]:
path = '/home/lavino/Documentos/projetos_indicium/ds_desafio/'

In [41]:
df_raw = pd.read_csv(path + 'data/df_fill.csv', index_col='year', parse_dates=True)
df_paramns = pd.read_csv(path + 'data/df_countrys_paramns.csv')

In [42]:
df_raw = df_raw.asfreq(freq='AS')

# 1.0 Predictions

In [43]:
df1 = df_raw.copy()

## 1.1 train test split

In [44]:
test_years = 5
train = df1[:-test_years]
test = df1[-test_years:]

## 1.2 train and predict for individual country

In [45]:
predict = pd.DataFrame()
for country in df1.columns:
    paramns = eval(df_paramns[df_paramns['country'] == country]['best_params'].values[0])
    model = SimpleExpSmoothing(
        train[country], 
        initialization_method=paramns['initialization_method']
    ).fit(
        smoothing_level=paramns['smoothing_level'], 
        optimized=False
    )
    
    forecast = model.forecast(len(test))
    
    predict = pd.concat([predict, pd.DataFrame(forecast, columns=[country])], axis=1)

## 1.3 plot and save predictions

In [46]:
# if not os.path.exists(path + 'images'):
#     os.makedirs(path + 'images')

# if not os.path.exists(path + 'images/predictions'):
#     os.makedirs(path + 'images/predictions')


# for country in df1.columns:

#     country_data = df_raw[country]

    

#     plt.figure(figsize=(10, 6))
#     df1[country].plot(legend=True,label=f'GDP {country}')
#     test[country].plot(legend=True,label='test')
#     predict[country].plot(label='predict',legend=True)
#     plt.suptitle(f'GDP prediction of {country} from 2024 to 2028', fontsize=16)
#     plt.tight_layout(rect=[0, 0.03, 1, 0.95])
#     plt.savefig(path + f'images/predictions/{country}_prediction.png')
#     plt.close()


## 1.4 Save dataframe

In [47]:
predict.to_csv(path + 'data/predict.csv', index=True)